# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [74]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [75]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/noorzaki/Desktop/module 6 challenge/python-api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City_ID.5,City_ID.4,City_ID.3,City_ID.2,City_ID.1,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,0,0,0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,1,1,1,1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,2,2,2,2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,3,3,3,3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,4,4,4,4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [76]:
city_data_df = city_data_df.drop(['City_ID.5','City_ID.4','City_ID.3','City_ID.2','City_ID.1'], axis=1)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [77]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 800,
    size= "Humditity",
    scale = 1,
    color = "City"
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [78]:
# Narrow down cities that fit criteria and drop any results with null values
#ideal_cities_df = city_data_df[(city_data_df['Max Temp']>=21) & (city_data_df['Max Temp']<=27)]& (city_data_df['Cloudiness']=0) & (city_data_df['Wind Speed (m/s)']<4.5)]
ideal_cities_df = city_data_df[(city_data_df['Max Temp']>=21) & (city_data_df['Max Temp']<=27) & (city_data_df['Cloudiness']==0) & (city_data_df['Wind Speed']<4.5)]
# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
print(ideal_cities_df)

     City_ID            City      Lat       Lng  Max Temp  Humidity  \
45        45           kapaa  22.0752 -159.3190     22.99        84   
51        51            hilo  19.7297 -155.0900     26.27        83   
63        63           banda  25.4833   80.3333     24.62        52   
81        81   makakilo city  21.3469 -158.0858     21.66        81   
152      152         kahului  20.8947 -156.4700     23.80        60   
197      197             gat  31.6100   34.7642     24.38       100   
211      211          laguna  38.4210 -121.4238     21.67        79   
240      240     tikaitnagar  26.9500   81.5833     23.56        59   
265      265     san quintin  30.4833 -115.9500     21.20        74   
340      340   santa rosalia  27.3167 -112.2833     24.62        56   
363      363          narwar  25.6500   77.9000     22.35        55   
375      375    port hedland -20.3167  118.5667     21.03        73   
381      381       roebourne -20.7833  117.1333     23.48        65   
391   

### Step 3: Create a new DataFrame called `hotel_df`.

In [79]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(HotelName='')

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,HotelName
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,
573,tatarsk,RU,55.2190,75.9828,89,
574,canchungo,GW,12.0672,-16.0333,61,
575,iquique,CL,-20.2208,-70.1431,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [80]:
# Set parameters to search for a hotel
radius = 10000
categories = "accomodation.hotel"
limit = 20
coordinates = f"{hotel_df.iloc[0]['Lat']},{hotel_df.iloc[0]['Lng']}"
#coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"

params = {
    'category':categories,
    'limit':limit,
    'apiKey':geoapify_key,
    'location': coordinates,
    'radius': radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat},{lng}"
    params["filter"] = f"circle:{lat},{lng},{radius}"
    #params["bias"] = f"{lat},{lng}"
    params["bias"] = f"proximity:{lat},{lng}"
    #params["location"] = f"{lat},{lng}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    #name_address = 
    name_address = requests.get(base_url, params = params).json()
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    #print(name_address)
    #name_address = name_address.json()
    
    #print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "HotelName"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "HotelName"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'HotelName']}")

# Display sample data
hotel_df

Starting hotel search
sovetskiy - nearest hotel: No hotel found
tatarsk - nearest hotel: No hotel found
canchungo - nearest hotel: No hotel found
iquique - nearest hotel: No hotel found
poum - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,HotelName
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,No hotel found
573,tatarsk,RU,55.2190,75.9828,89,No hotel found
574,canchungo,GW,12.0672,-16.0333,61,No hotel found
575,iquique,CL,-20.2208,-70.1431,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [81]:
%%capture --no-display

# Configure the map plot
hotel_map_add = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 900,
    frame_height = 800,
    tiles = "OSM",
    size = "Humidity",
    scale= 1,
    color = "City",
    hover_cols = ["Country","Hotel Name" ]
)

# Display the map
hotel_map_add 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)